# Estimando modelos lineares

In [1]:
import statsmodels.api as sm 
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd

In [2]:
# Gerar um modelo linear a partir de alguns dados aleatórios
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * np.random.randn(*size)

In [3]:
# Para possibilitar uma reprodução
rng = np.random.default_rng(seed=12345)

N = 100
X = np.c_[dnorm(0, 0.4, size=N),
          dnorm(0, 0.6, size=N),
          dnorm(0, 0.2, size=N)]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

In [4]:
# Mostrando X
X[:5]

array([[-0.10097656,  0.92698709,  0.63699036],
       [-0.4617771 , -0.27960806, -0.27622907],
       [-0.60732433,  1.44241868, -0.2832296 ],
       [-1.93295274, -0.17316474, -0.00719193],
       [-0.81006057,  0.59728404, -0.45368153]])

In [5]:
# Mostrando y
y[:5]

array([ 0.3731393 , -0.68513577,  0.42434282, -0.28238359, -0.17729788])

In [6]:
# Adicionar uma coluna de uns a um array
X_model = sm.add_constant(X)
X_model[:5]

array([[ 1.        , -0.10097656,  0.92698709,  0.63699036],
       [ 1.        , -0.4617771 , -0.27960806, -0.27622907],
       [ 1.        , -0.60732433,  1.44241868, -0.2832296 ],
       [ 1.        , -1.93295274, -0.17316474, -0.00719193],
       [ 1.        , -0.81006057,  0.59728404, -0.45368153]])

In [7]:
# A função sm.add_constant pode adicionar uma coluna de interceptação em uma matriz existente
model = sm.OLS(y, X)

In [8]:
# O método fit do modelo devolve um objeto de resultado da regressão contendo os parâmetros estimados do 
# modelo e outros diagnósticos
results = model.fit()
results.params


array([0.02087758, 0.32334906, 0.3880625 ])

In [9]:
# O método summary em results é capaz de exibir uma saída de diagnóstico do detalhadamente do modelo
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.602
Model:                            OLS   Adj. R-squared (uncentered):              0.589
Method:                 Least Squares   F-statistic:                              48.87
Date:                Fri, 14 Apr 2023   Prob (F-statistic):                    2.49e-19
Time:                        12:35:25   Log-Likelihood:                         -6.9233
No. Observations:                 100   AIC:                                      19.85
Df Residuals:                      97   BIC:                                      27.66
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [10]:
# Novo DataFrame, e adicionando a coluna y
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])

data['y'] = y

data[:5]

,col0,col1,col2,y
0,-0.100977,0.926987,0.636990,0.373139
1,-0.461777,-0.279608,-0.276229,-0.685136
2,-0.607324,1.442419,-0.283230,0.424343
3,-1.932953,-0.173165,-0.007192,-0.282384
4,-0.810061,0.597284,-0.453682,-0.177298


In [11]:
# Usar a API de fórmulas do statsmodels e as strings de fórmulas do Patsy
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()

results.params

Intercept    0.015013
col0         0.020697
col1         0.323955
col2         0.383970
dtype: float64

In [12]:
# Usando a propriedade tvalues
results.tvalues

Intercept    0.564034
col0         0.564876
col1         9.932795
col2         6.214569
dtype: float64

In [13]:
# Podemos calcular os valores previstos, considerando os parâmetros estimados pelo modelo
results.predict(data[:5])

0    0.557810
1   -0.191189
2    0.360971
3   -0.083852
4    0.017540
dtype: float64

In [14]:
# Simular alguns dados de séries temporais com uma estrutura autorregressiva e ruído
init_x = 4

values = [init_x, init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

In [15]:
# Podemos fazer a adequação do modelo com um número maior de defasagens
from statsmodels.tsa.ar_model import AutoReg
MAXLAGS = 5
model = AutoReg(values, MAXLAGS)
results = model.fit()

In [16]:
# Os parâmetros estimados no resultado têm a intercepção antes e as estimativas para as duas primeiras defasagens
results.params

array([-0.00948485,  0.80974801, -0.44095373,  0.05148024, -0.0563773 ,
        0.02984335])